# Computing Errors of Pose Prediction from PvNet

This notebook is to walk through the process of computing errors of pose prediction result from PvNet, including translation error and angular(rotation) error.

Let's say we have a set of predicted poses represented by transformation matrices: 
$$\{{T_{pre}}\}_i = \begin{bmatrix} R_{pre}& t_{pre} \\ 0&1 \end{bmatrix}_i $$
where $R_{pre}$ is the 3x3 rotation matrix representing the predicted rotation, and $t_{pre}$ is the 3x1 translation vector representing the predicted translation of the $i$-th pose.

Similarly, we have a set of corresponding ground truth poses represented by transformation matrices:
$$\{{T_{gt}}\}_i = \begin{bmatrix} R_{gt}& t_{gt} \\ 0&1 \end{bmatrix}_i $$

## 1. Translation Error

### 1.1 Method
The translation error can be calculated as the Euclidean distance between $t_{pre}=(x_1, y_1, z_1)$ and $t_{gt}=(x_2, y_2, z_2)$. 

Mathematically, the translation error for each dimension is given by:
$$
\text{x\_error}_{i} = | x_1 - x_2 |
$$
$$
\text{y\_error}_{i} = | y_1 - y_2 |
$$
$$
\text{z\_error}_{i} = | z_1 - z_2 |
$$

#### Implementation

In [5]:
import numpy as np

def translation_error(T_pre, T_gt):
    t_pre = T_pre[:3, 3]
    t_gt = T_gt[:3, 3]
    translation_error = np.abs(t_pre - t_gt)
    return translation_error

#### Example

In [6]:
T_gt = np.array([
    [-0.9992905, -0.02776036, 0.02545223, -0.0097374 ],
    [-0.01929233, 0.95770321, 0.2871104,   0.01518335],
    [-0.03234596, 0.2864157, -0.95755938,  1.51049271],
    [0., 0., 0., 1.]
])

T_pre = np.array([
    [-0.9995757, -0.02325838,  0.0175349, -0.00948184],
    [-0.01803212, 0.96688755,  0.25456493, 0.014642  ],
    [-0.02287504, 0.25414072, -0.9668967,  1.46206129],
    [0., 0., 0., 1.]
])

In [7]:
t_error = translation_error(T_pre, T_gt)
print(f"x error = {t_error[0]*1000:.2f} mm")
print(f"y error = {t_error[1]*1000:.2f} mm")
print(f"z error = {t_error[2]*1000:.2f} mm")

x error = 0.26 mm
y error = 0.54 mm
z error = 48.43 mm


## 2. Angular Error

### 2.1 Method1 - Using rotation matrices
The angular error is the difference between the predicted rotation $R_{pre}$ and $R_{gt}$. Mathematically,

$$R_{gt}^{pre} = R_{world}^{pre} R_{gt}^{world} = R_{world}^{pre} (R_{world}^{gt})^T$$
$$\theta = \cos^{-1} \left(\dfrac{\text{trace}\left( R_{gt}^{pre} \right) - 1}{2}\right) $$

Reference: 

[1. Axis–angle representation - WIKIPEDIA](https://en.wikipedia.org/wiki/Axis%E2%80%93angle_representation#Exponential_map_from_so.283.29_to_SO.283.29)

[2. Distance between rotations - Blog](http://www.boris-belousov.net/2016/12/01/quat-dist/#using-quaternions)

Implementation

In [13]:
def angular_R_error(T_pre, T_gt):
    R_pre_in_world = T_pre[:3, :3]
    R_gt_in_world = T_gt[:3, :3]
    R_pre_2_gt = np.dot(R_pre_in_world, R_gt_in_world.T)
    print(f"R_pre_2_gt = \n{R_pre_2_gt}")
    trace = np.trace(R_pre_2_gt)
    trace = trace if trace <= 3 else 3
    trace = trace if trace >= -1 else -1
    angular_distance = np.rad2deg(np.arccos((trace - 1.) / 2.))
    return angular_distance

In [14]:
angular_R_err = angular_R_error(T_pre, T_gt)
print(f"Angular Error computed by R matrices = {angular_R_err:.2f} deg")

R_pre_2_gt = 
[[ 0.99995846  0.00204397  0.00887996]
 [-0.00234258  0.99942743  0.033754  ]
 [-0.0088059  -0.0337734   0.99939081]]
Angular Error computed by R matrices = 2.00 deg


### 2.2 Method2 - Using Quaternions
Firstly, convert rotation matrices to quaternion representation.
The angular error is the difference between the predicted rotation $R_{pre}$ and $R_{gt}$. Mathematically,

$$R_{gt}^{pre} = R_{world}^{pre} R_{gt}^{world} = R_{world}^{pre} (R_{world}^{gt})^T$$
$$\theta = \cos^{-1} \left(\dfrac{\text{trace}\left( R_{gt}^{pre} \right) - 1}{2}\right) $$

Reference: 

[1. Axis–angle representation - WIKIPEDIA](https://en.wikipedia.org/wiki/Axis%E2%80%93angle_representation#Exponential_map_from_so.283.29_to_SO.283.29)

[2. Distance between rotations - Blog](http://www.boris-belousov.net/2016/12/01/quat-dist/#using-quaternions)

In [2]:
import os
import numpy as np
path = "/home/yangfei/clean-pvnet/data"
path_trans = os.path.join(path, "trans_err.npy")
trans_err = np.load(path_trans)
print(len(trans_err))
print(trans_err)

path_R = os.path.join(path, "ang_R_err.npy")
R_err = np.load(path_R)
print(len(R_err))
print(R_err)

path_Q = os.path.join(path, "ang_Q_err.npy")
Q_err = np.load(path_Q)
print(len(Q_err))
print(Q_err)

2000
[[4.27858208e-01 9.37207832e-01 5.80340688e+01]
 [2.29019414e-01 5.05099851e-01 4.51141417e+01]
 [9.56568425e-01 1.40319250e-01 7.51563556e+01]
 ...
 [2.00240502e+00 6.92373054e-01 1.02820756e+02]
 [5.78667867e-01 5.79090619e-02 5.09114359e+01]
 [3.64594402e-01 6.81226554e-01 5.05859588e+01]]
2000
[1.45040639 1.99054171 1.10031806 ... 3.14390361 1.36634796 2.10326126]
2000
[1.45055172 1.99060406 1.10010096 ... 3.14391362 1.36644833 2.10340228]


In [1]:
import numpy as np

#### "Reasonable"

In [2]:
T_1in0 = np.array([
    [-0.9995757, -0.02325838,  0.0175349, -0.00948184],
    [-0.01803212, 0.96688755,  0.25456493, 0.014642  ],
    [-0.02287504, 0.25414072, -0.9668967,  1.46206129],
    [0., 0., 0., 1.],
])

T_2in0 = np.array([
    [-0.9992905, -0.02776036, 0.02545223, -0.0097374 ],
    [-0.01929233, 0.95770321, 0.2871104,   0.01518335],
    [-0.03234596, 0.2864157, -0.95755938,  1.51049271],
    [0., 0., 0., 1.],
])

R_1in0 = T_1in0[0:3, 0:3]
R_2in0 = T_2in0[0:3, 0:3]

print(R_1in0)
print(R_2in0)

[[-0.9995757  -0.02325838  0.0175349 ]
 [-0.01803212  0.96688755  0.25456493]
 [-0.02287504  0.25414072 -0.9668967 ]]
[[-0.9992905  -0.02776036  0.02545223]
 [-0.01929233  0.95770321  0.2871104 ]
 [-0.03234596  0.2864157  -0.95755938]]


First check at least the the determinant of both rotation matrices is $1$.

In [3]:
print(f'det(R_1in0) = {np.linalg.det(R_1in0):6.3f}')
print(f'det(R_2in0) = {np.linalg.det(R_2in0):6.3f}')

det(R_1in0) =  1.000
det(R_2in0) =  1.000


$$R_2^1 = R_0^1 R_2^0 = (R_1^0)^T R_2^0$$

In [4]:
R_2in1 = R_1in0.T @ R_2in0

print(R_2in1)

[[ 0.9999543   0.00392739 -0.00871443]
 [-0.00363206  0.99942686  0.03365666]
 [ 0.00884159 -0.03362352  0.99939555]]


$$\theta = \cos^{-1} \left(\dfrac{\text{trace}\left( R_2^1 \right) - 1}{2}\right) $$

In [5]:
theta_2in1 = np.arccos((R_2in1.trace() - 1.) / 2.)

print(f'theta_2in1 = {theta_2in1:6.3f} rad = {np.rad2deg(theta_2in1):6.3f} deg')

theta_2in1 =  0.035 rad =  2.004 deg


#### "Unreasonable"

In [6]:
T_1in0 = np.array([
    [ 0.50412449, -0.86339407, -0.02022804, 0.00188722],
    [ 0.85312843, 0.49421558, 0.1671013, -0.01022479],
    [ 0.13427726, 0.1014969, -0.98573221, 1.72247981],
    [0., 0., 0., 1.],
])

T_2in0 = np.array([
    [ 0.51519562, -0.85650104, -0.03129786, 0.00203341],
    [ 0.85454693, 0.51053328, 0.09542202, -0.01044806],
    [-0.06575047, -0.07590649, 0.99494484, 1.774678],
    [0., 0., 0., 1.],
])

R_1in0 = T_1in0[0:3, 0:3]
R_2in0 = T_2in0[0:3, 0:3]

print(R_1in0)
print(R_2in0)

[[ 0.50412449 -0.86339407 -0.02022804]
 [ 0.85312843  0.49421558  0.1671013 ]
 [ 0.13427726  0.1014969  -0.98573221]]
[[ 0.51519562 -0.85650104 -0.03129786]
 [ 0.85454693  0.51053328  0.09542202]
 [-0.06575047 -0.07590649  0.99494484]]


$$R_2^1 = R_0^1 R_2^0 = R_0^1 (R_0^2)^T$$

In [7]:
R_2in1 = R_1in0.T @ R_2in0

print(R_2in1)

[[ 0.97993222 -0.00642521  0.19922769]
 [-0.02915991  0.98410715  0.17516525]
 [ 0.19718686  0.17745958 -0.96417094]]


$$\theta = \cos^{-1} \left(\dfrac{\text{trace}\left( R_2^1 \right) - 1}{2}\right) $$

In [8]:
theta_2in1 = np.arccos((R_2in1.trace() - 1.) / 2.)

print(f'theta_2in1 = {theta_2in1:6.3f} rad = {np.rad2deg(theta_2in1):6.3f} deg')

theta_2in1 =  1.571 rad = 90.004 deg


We think this result is wrong. First, let's check if $R_1^0$ and $R_2^0$ really are rotation matrices. Here's the first condition these matrices have to satisfy:

$$\det(R_1^0) = \det(R_2^0) = 1$$

In [9]:
print(f'det(R_1in0) = {np.linalg.det(R_1in0):6.3f}')
print(f'det(R_2in0) = {np.linalg.det(R_2in0):6.3f}')

det(R_1in0) = -1.000
det(R_2in0) =  1.000


The problem, clearly, is that $R_1^0$ is not a valid rotation matrix --- in particular, it satisfies the "left-hand rule" rather than the "right-hand rule."

In [10]:
print(f'det(-R_1in0) = {np.linalg.det(-R_1in0):6.3f}')

det(-R_1in0) =  1.000


#### Test the "Rodrigues" function from OpenCV

We want to know why the following code is producing bad results (invalid rotation matrices):

```
def pnp(points_3d, points_2d, camera_matrix, method=cv2.SOLVEPNP_ITERATIVE):
    try:
        dist_coeffs = pnp.dist_coeffs
    except:
        dist_coeffs = np.zeros(shape=[8, 1], dtype='float64')

    assert points_3d.shape[0] == points_2d.shape[0], 'points 3D and points 2D must have same number of vertices'
    if method == cv2.SOLVEPNP_EPNP:
        points_3d = np.expand_dims(points_3d, 0)
        points_2d = np.expand_dims(points_2d, 0)

    points_2d = np.ascontiguousarray(points_2d.astype(np.float64))
    points_3d = np.ascontiguousarray(points_3d.astype(np.float64))
    camera_matrix = camera_matrix.astype(np.float64)
    _, R_exp, t = cv2.solvePnP(points_3d,
                               points_2d,
                               camera_matrix,
                               dist_coeffs,
                               flags=method)
    # , None, None, False, cv2.SOLVEPNP_UPNP)

    # R_exp, t, _ = cv2.solvePnPRansac(points_3D,
    #                            points_2D,
    #                            cameraMatrix,
    #                            distCoeffs,
    #                            reprojectionError=12.0)

    R, _ = cv2.Rodrigues(R_exp)
    # trans_3d=np.matmul(points_3d,R.transpose())+t.transpose()
    # if np.max(trans_3d[:,2]<0):
    #     R=-R
    #     t=-t

    return np.concatenate([R, t], axis=-1)
```

In [11]:
import cv2
print("OpenCV (cv2) version:", cv2.__version__)

OpenCV (cv2) version: 4.5.5


In [12]:
rvec = np.array([1., 0., 0.])
[R, J] = cv2.Rodrigues(rvec)
print(R)
print(np.linalg.det(R))
print(np.allclose(R.T @ R, np.eye(3)))

[[ 1.          0.          0.        ]
 [ 0.          0.54030231 -0.84147098]
 [ 0.          0.84147098  0.54030231]]
1.0
True


In [13]:
rvec = np.array([0., 0., 0.])
[R, J] = cv2.Rodrigues(rvec)
print(R)
print(np.linalg.det(R))
print(np.allclose(R.T @ R, np.eye(3)))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
1.0
True


In [14]:
maxiter = 1000
for iter in range(maxiter):
    if not iter % 100:
        print(f'{iter:05d} / {maxiter:05d}')
    rvec = 10. * np.random.randn(3)
    [R, J] = cv2.Rodrigues(rvec)
    detR = np.linalg.det(R)
    if not np.isclose(detR, 1.):
        print(f'bad determinant: {detR:6.3f}')
        print(rvec)
        print(R)
        print(J)
        break
    if not np.allclose(R.T @ R, np.eye(3)):
        print(f'bad axes')
        print(rvec)
        print(R)
        print(J)
        break

00000 / 01000
00100 / 01000
00200 / 01000
00300 / 01000
00400 / 01000
00500 / 01000
00600 / 01000
00700 / 01000
00800 / 01000
00900 / 01000


Some conclusions (discussed this more in person):

* The order of outputs of `cv2.solvePnP` might be wrong.
* The code that you claimed to use must not be the code you actually used (probably), because `cv2.Rodrigues` should --- according to our results --- *always* produce a valid rotation matrix.

In [15]:
# Opencv version in PvNet codebase = '3.4.2'

In [16]:
T_1in0 = np.array([[-0.50412625, 0.86339166, 0.02028718, -0.00188714],
                    [-0.85312665, -0.49420528, -0.16714087, 0.01022473],
                    [-0.134282, -0.10156763, 0.98572428, -1.72247363]])


T_2in0 = np.array([[0.51519562, -0.85650104, -0.03129786, 0.00203341],
                    [0.85454693, 0.51053328, 0.09542202, -0.01044806],
                    [-0.06575047, -0.07590649, 0.99494484, 1.774678]])


R_1in0 = T_1in0[0:3, 0:3]
R_2in0 = T_2in0[0:3, 0:3]

print(R_1in0)
print(R_2in0)


R_2in1 = R_1in0.T @ R_2in0

print(R_2in1)

theta_2in1 = np.arccos((R_2in1.trace() - 1.) / 2.)
print(f'theta_2in1 = {theta_2in1:6.3f} rad = {np.rad2deg(theta_2in1):6.3f} deg')

print(f'det(R_1in0) = {np.linalg.det(R_1in0):6.3f}')
print(f'det(R_2in0) = {np.linalg.det(R_2in0):6.3f}')

[[-0.50412625  0.86339166  0.02028718]
 [-0.85312665 -0.49420528 -0.16714087]
 [-0.134282   -0.10156763  0.98572428]]
[[ 0.51519562 -0.85650104 -0.03129786]
 [ 0.85454693  0.51053328  0.09542202]
 [-0.06575047 -0.07590649  0.99494484]]
[[-0.97993129  0.00642799 -0.19923218]
 [ 0.02917212 -0.98409446 -0.17523457]
 [-0.19718969 -0.17752984  0.96415742]]
theta_2in1 =  3.130 rad = 179.343 deg
det(R_1in0) =  1.000
det(R_2in0) =  1.000
